# GenAI Document Processing: Expense processing:

Based on the AWS Blog: https://aws.amazon.com/blogs/machine-learning/intelligent-document-processing-using-amazon-bedrock-and-anthropic-claude/

In [2]:
import boto3
import json
import base64

In [3]:
s3 = boto3.client('s3', region_name='us-east-1')
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"


In [8]:
# image_data = s3.get_object(Bucket=bucket_name, Key=object_key)['Body'].read()

image_file = "./test_invoice.png"

with open(image_file, "rb") as image:
  img = image.read()
  img_bytes = bytearray(img)

base64_image = base64.b64encode(img_bytes).decode('utf-8')


In [9]:
# prompt:

prompt = """
This image shows a an invoice. 
Please precisely copy all the relevant information from the form.
Leave the field blank if there is no information in corresponding field.
If the image is not an invoice, simply return an empty JSON object. 
If the invoice is not filled, leave the fees attributes blank. 
Translate any non-English text to English. 
Organize and return the extracted data in a JSON format with the following keys:
{
    "InvoiceDetails":{
        "invoiceNum": "",
        "date": "",
        "billToAddress": "",
        "city": "",
        "state": "",
        "zipCode": "",
        "phone": "",
        "email":""
    },
    "items":[{
        "ItemDescription": "",
        "quantity": "",
        "unitPrice": "",
        "amount": "",
    }
    ],
    "total": "",
    "currency": ""
  }
""" 



In [10]:
def invoke_claude_3_multimodal(prompt, base64_image_data):
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 2048,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": base64_image_data,
                        },
                    },
                ],
            }
        ],
    }

    try:
        response = bedrock.invoke_model(modelId=MODEL_ID, body=json.dumps(request_body))
        return json.loads(response['body'].read())
    except bedrock.exceptions.ClientError as err:
        print(f"Couldn't invoke Claude 3 Sonnet. Here's why: {err.response['Error']['Code']}: {err.response['Error']['Message']}")
        raise


In [11]:
response = invoke_claude_3_multimodal(prompt, base64_image)

In [12]:
response

{'id': 'msg_bdrk_01Yazczd9NPrfmptKiFTy5CC',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-sonnet-20240229',
 'content': [{'type': 'text',
   'text': '{\n    "InvoiceDetails": {\n        "invoiceNum": "2170",\n        "date": "2017/09/29",\n        "billToAddress": "Mellicent Ivoshin Dynazzy 37 Carpenter Court Sinilian First",\n        "city": "",\n        "state": "",\n        "zipCode": "560-390-2703",\n        "phone": "",\n        "email": "mivoshincp@gravatar.com"\n    },\n    "items": [\n        {\n            "ItemDescription": "Service Fee",\n            "quantity": "6",\n            "unitPrice": "200.00",\n            "amount": "1 200.00"\n        },\n        {\n            "ItemDescription": "Additional Services", \n            "quantity": "7",\n            "unitPrice": "75.00",\n            "amount": "525.00"\n        }\n    ],\n    "total": "1 725.00",\n    "currency": "$"\n}'}],
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'usage': {'input_tokens': 1

In [13]:
print(response["content"][0]["text"])

{
    "InvoiceDetails": {
        "invoiceNum": "2170",
        "date": "2017/09/29",
        "billToAddress": "Mellicent Ivoshin Dynazzy 37 Carpenter Court Sinilian First",
        "city": "",
        "state": "",
        "zipCode": "560-390-2703",
        "phone": "",
        "email": "mivoshincp@gravatar.com"
    },
    "items": [
        {
            "ItemDescription": "Service Fee",
            "quantity": "6",
            "unitPrice": "200.00",
            "amount": "1 200.00"
        },
        {
            "ItemDescription": "Additional Services", 
            "quantity": "7",
            "unitPrice": "75.00",
            "amount": "525.00"
        }
    ],
    "total": "1 725.00",
    "currency": "$"
}
